In [1]:
%reload_ext autoreload
%autoreload 2

# %matplotlib widget
%gui qt

import time
import random
from copy import copy
from tqdm import tqdm
from pathlib import Path
import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.width = 1000

from vrAnalysis import session
from vrAnalysis import functions
from vrAnalysis import analysis
from vrAnalysis import helpers
from vrAnalysis import fileManagement as fm
from vrAnalysis import database
from vrAnalysis import tracking

from vrAnalysis.redgui import redCellGUI as rgui

vrdb = database.vrDatabase()

(CVXPY) Oct 19 01:33:37 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


In [125]:
# Immediate next steps for programming work:
# - ROICaT index alignment tools

In [241]:
# Database Requirements: 
# ---------------------
# GUI: db manager
# - show full database -
# - show selected entries (like for example entries with imaging but no suite2p) -
#                --> It would be sweet to make this work where there are some defaults, but then there is also a "filter command" option which you type and goes as the input to the filtering
# - click on entry and do things:
#                --> open file explorer to that session
#                --> do suite2p
#                --> do red cell management
# - update table data? 
# ---------------------
# Operational Commands: 
# - Automatically do suite2p 
# - Check if registration was done before a suite2p update
# - Find way to determine if a suite2p file was updated after the folder was made (indicating that QC was done afterwards) -- so registration should be rerun for imaging data at least
#    -- check if "iscell" is the only thing that was done after registration. In this case, just update the iscell oneData
# ---------------------
# - Add new entry GUI!!!
# ---------------------
# - Check session folders to see if new session is present that isn't registered in the database? 
# ---------------------
# vrExperimentUpdates: 
# -- if edited iscell, will need to re-register the number of ROIs per plane (in "self.values") and reprocess imaging data!
# ---------------------
# vrExperimentRegistration
# -- if error, then note there was an error and append an explanation...!!!
# ---------------------

In [2]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
sessionid = '701'
# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In [19]:
# I need to remove the silly variables from CR_Hippos on D:/

In [26]:
# ROICaT Alignment Tools
# Track Red Cell Consistency across days

In [99]:
track = tracking.tracker('ATL022')

In [100]:
ucids = [[[] for _ in range(track.num_sessions)] for _ in range(track.num_planes)]

In [109]:
for planeidx, results in enumerate(track.results):
    for sesidx, labels in enumerate(results['clusters']['labels_bySession']):
        ucids[planeidx][sesidx] = labels

In [42]:
track.results[0]['clusters'].keys()

dict_keys(['labels', 'labels_bySession', 'labels_bool', 'labels_bool_bySession', 'labels_dict'])

In [106]:
track.results[0]['clusters']['labels']

array([  -1,   -1,   -1, ..., 6972,   -1,   -1], dtype=int64)

In [ ]:
# %% -- And Load ROICaT Files --

# roicatPath = fullfile(cdData,mouseName,'roicat');
# roicatFiles = dir(fullfile(roicatPath,'*.mat'));
# assert(length(roicatFiles)==numPlanes, "If this isn't true, hopefully it's just labeled wrong...");
# roicat = cell(numPlanes, 1);
# roicatUCIDs = cell(numPlanes, NS);
# roicatPlane = zeros(numPlanes,1);
# for np = 1:numPlanes
#     roicat{np} = load(fullfile(roicatPath, roicatFiles(np).name));
#     roicatPlane(np) = str2double(roicat{np}.META.usePlane(6:end));
#     roicatUCIDs(np,:) = roicat{np}.UCIDs_bySession;
# end
# [~,planeIdx] = sort(roicatPlane);
# roicat = roicat(planeIdx);
# clear('roicatPlane','planeIdx')

# % Create Useful ROICaT Indices
# maxUCID = max(cellfun(@max, roicatUCIDs),[],2);
# roicatIndex = cell(numPlanes,1);
# for np = 1:numPlanes
#     roicatIndex{np} = zeros(maxUCID(np),NS);
#     for ns = 1:NS
#         cindex = roicatUCIDs{np,ns}(roicatUCIDs{np,ns}>0); % Get list of UCIDs (sort doesn't matter)
#         roicatIndex{np}(cindex,ns) = 1; % label presence of UCID as 1
#     end
# end

# % Sanity check
# anyNotEqual = false;
# for ns = 1:NS
#     for np = 1:numPlanes
#         cellsInPlane = sum(imData{ns}.planeIdx==np-1);
#         cellsInROICaT = length(roicat{np}.UCIDs_bySession{ns});
#         if cellsInPlane ~= cellsInROICaT
#             fprintf(1, 'Session %i, Plane %i: ROIs: %i, ROICaT: %i -- not equal!!!\n', ns,np,cellsInPlane,cellsInROICaT);
#             anyNotEqual = true; 
#         end
#     end
# end
# if anyNotEqual, error('Look at above messages, something went wrong!'); end

# useSessions = 1:6;
# cellsPerPlane = num2cell(cumsum([zeros(1,length(useSessions)); cell2mat(arrayfun(@(planeIdx) cellfun(@(imData) sum(imData.planeIdx==planeIdx), imData(useSessions), 'uni', 1), (0:3)', 'uni', 0))],1)); % used as offset for indices
# idxInUseSessions = cellfun(@(roicatIndex) find(all(roicatIndex(:,useSessions),2)), roicatIndex, 'uni', 0); % list of UCIDs that are in all requested sessions
# [~,idxToUCID] = cellfun(@(ucid,idx) ismember(idx,ucid), roicatUCIDs(:,useSessions), repmat(idxInUseSessions,1,length(useSessions)), 'uni', 0);
# masterIdx = cell2mat(cellfun(@(idx,offset) idx + offset, idxToUCID, cellsPerPlane, 'uni', 0));
# NUS = length(useSessions);

In [ ]:
# next step in analysis: 
# Also - using a cutoff point by the hitch in that distribution might ignore true matches that are soma-dendritic, but maybe I can use ROICaT or npix to avoid those? 

In [ ]:
# okay, looking at these clusters, it seems like there is some serious oversplitting going on in suite2p. That sucks!
# I think I should probably try to rerun some suite2p sessions....
# But for now I'll probably do a "merge" procedure... and I honestly think I'll have to overwrite or add a new "modified" suite2p folder with the stat and ops files